# WEB SCRAPPING
## Data mining from Peruvian Open Source Data content provided by the Goverment

### Scope of this Notebook
[Datos Abiertos]("https://www.datosabiertos.gob.pe") is a raw open data source provide by the goverment of Peru.
I used webscrapping methods in **Python** to obtain relevant data from a *energy* sector.
At the end of the nombre, a dataset is provided containing the results of the webscrapping and data mining.
This is just a taste of what I am able to do for collecting data from web pages.

In [2]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#data files in energy tab
data_files = 35
n_links_per_page = 10
n_pages = int(np.ceil(data_files / n_links_per_page))

print(f"The category ENERGY has {data_files} data files")
print(f"Only {n_links_per_page} data links are shown per page")
print(f"Therefore, all the links are in a total of {n_pages}")

The category ENERGY has 35 data files
Only 10 data links are shown per page
Therefore, all the links are in a total of 4


In [4]:
# For page 1 with 10 results (35 results for Energy, 4 browsing pages)
# getting all URLs
URL = []
main_web = "https://www.datosabiertos.gob.pe"
field_web = "/search/field_topic/energ%C3%ADa-340"

for i in range(n_pages):
    
    query = f"?query=&sort_by=changed&sort_order=DESC&page=0%2C{i}"
    URL.append(main_web + field_web + query)

In [5]:
children = []

for url in URL:
    data  = requests.get(url).text
    soup = BeautifulSoup(data, "html.parser")
    tag_view_content = soup.body.find_all(class_="view-content")[0]

    for child in tag_view_content.children:
        if child != '\n':
            children.append(child)

In [6]:
url = "https://www.datosabiertos.gob.pe/search/field_topic/energ%C3%ADa-340?query=&sort_by=changed&sort_order=DESC&page=0%2C0"
data  = requests.get(url).text
soup = BeautifulSoup(data, "html.parser")  # create a soup object using the variable 'data'
tag_view_content = soup.body.find_all(class_="view-content")[0]

In [7]:
print(f"The total number of files to be download are: {len(children)}")

The total number of files to be download are: 35


In [8]:
df_meta_data = pd.DataFrame(columns=["title", "link", "produced_by", "description",])

for i, child in enumerate(children):    
    item = {}
    
    if not child.find_all(class_="node-title"):
        item["title"] = None
    else:
        item["title"] = child.find_all(class_="node-title")[0].text
        
    if not [child.h2.a["href"]]:
        item["link"] = None
    else:
        item["link"] = main_web + child.h2.a["href"]
        
    if not child.find_all(class_="group-membership"):
        item["produced_by"] = None
    else:
        item["produced_by"] = child.find_all(class_="group-membership")[0].text
    
    if not child.find_all(class_="node-description")[0].p:
        item["description"] = None
    else:
        item["description"] = child.find_all(class_="node-description")[0].p.text
    
    df_meta_data = pd.concat([df_meta_data, pd.DataFrame([item])], ignore_index=True)
    
del soup

In [9]:
df_meta_data.head()

,title,link,produced_by,description
0,Consumo de Energía Eléctrica de los clientes d...,https://www.datosabiertos.gob.pe/dataset/consu...,Empresa de Administración en Infraestructura E...,Consumo de Energía Eléctrica de los clientes d...
1,Consumo de Energía Eléctrica de los clientes d...,https://www.datosabiertos.gob.pe/dataset/consu...,Electro Ucayali,None
2,GASTOS DE USO DE VEHÍCULOS DE LA EMPRESA ELECT...,https://www.datosabiertos.gob.pe/dataset/gasto...,Electro Ucayali,GASTOS DE USO DE VEHÍCULOS DE LA EMPRESA ELECT...
3,ORDENES DE COMPRA Y SERVICIOS DE LA EMPRESA EL...,https://www.datosabiertos.gob.pe/dataset/orden...,Electro Ucayali,Es el registro de órdenes de compra y servicio...
4,Registro de Hidrocarburos Líquidos - Setiembre...,https://www.datosabiertos.gob.pe/dataset/regis...,Presidencia del Consejo de Ministros,Para desarrollar actividades de comercializaci...


In [10]:
# Getting the missing labels for the meta data of each file
# They are extracted from the specific page of each dataset inside the energy sector
labels = []
for link in df_meta_data["link"]:    
    data = requests.get(link).text
    soup = BeautifulSoup(data, "html.parser")  # create a soup object using the variable 'data'
    
    for label in soup.table.find_all(class_="field-label"):
        labels.append(label.text)

In [11]:
labels = list(set(labels))

In [12]:
df_meta_data = pd.concat([df_meta_data, pd.DataFrame(None, index=range(data_files), columns=list(labels))], axis=1)
df_meta_data.head(3)

,title,link,produced_by,description,Fecha de lanzamiento,Frequency,Publisher,Identificador,Language,Contact Email,Temporal Coverage,Fecha modificada,Author,License,Public Access Level,Homepage URL,Contact Name
0,Consumo de Energía Eléctrica de los clientes d...,https://www.datosabiertos.gob.pe/dataset/consu...,Empresa de Administración en Infraestructura E...,Consumo de Energía Eléctrica de los clientes d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Consumo de Energía Eléctrica de los clientes d...,https://www.datosabiertos.gob.pe/dataset/consu...,Electro Ucayali,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GASTOS DE USO DE VEHÍCULOS DE LA EMPRESA ELECT...,https://www.datosabiertos.gob.pe/dataset/gasto...,Electro Ucayali,GASTOS DE USO DE VEHÍCULOS DE LA EMPRESA ELECT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
for n, row in df_meta_data.iterrows():
    print(f"Row N°: {n}")
    print(row["link"])
    data = requests.get(row["link"]).text
    soup = BeautifulSoup(data, "html.parser")  # create a soup object using the variable 'data'
    table = soup.find(class_="field-group-format group_additional")
    for i, tr in enumerate(table.find_all("tr")):
        if i == 0:
            continue
  
        for label in labels:
            if label in tr.text:
                #print(label)
                #print("\t", tr.text.split(label)[1])
                df_meta_data.at[n, label] = tr.text.split(label)[1]
            else:
                continue


Row N°: 0
https://www.datosabiertos.gob.pe/dataset/consumo-de-energ%C3%ADa-el%C3%A9ctrica-de-los-clientes-de-adinelsa-setiembre-2023
Row N°: 1
https://www.datosabiertos.gob.pe/dataset/consumo-de-energ%C3%ADa-el%C3%A9ctrica-de-los-clientes-de-electro-ucayali-sa
Row N°: 2
https://www.datosabiertos.gob.pe/dataset/gastos-de-uso-de-veh%C3%ADculos-de-la-empresa-electro-ucayali-sa-electro-ucayali-sa-eluc
Row N°: 3
https://www.datosabiertos.gob.pe/dataset/ordenes-de-compra-y-servicios-de-la-empresa-electro-ucayali-sa-electro-ucayali-sa-eluc
Row N°: 4
https://www.datosabiertos.gob.pe/dataset/registro-de-hidrocarburos-l%C3%ADquidos-setiembre-2023-osinergmin
Row N°: 5
https://www.datosabiertos.gob.pe/dataset/consumo-de-energ%C3%ADa-el%C3%A9ctrica-de-los-clientes-de-adinelsa-agosto-2023
Row N°: 6
https://www.datosabiertos.gob.pe/dataset/consumo-energ%C3%A9tico-de-clientes-hidrandina-distriluz-dlz
Row N°: 7
https://www.datosabiertos.gob.pe/dataset/consumo-de-energ%C3%ADa-el%C3%A9ctrica-de-los-clien

In [14]:
df_meta_data.to_pickle("./energy_metadata.pkl")

In [15]:
import pandas as pd

df = pd.read_pickle("energy_metadata.pkl")

df.head()

,title,link,produced_by,description,Fecha de lanzamiento,Frequency,Publisher,Identificador,Language,Contact Email,Temporal Coverage,Fecha modificada,Author,License,Public Access Level,Homepage URL,Contact Name
0,Consumo de Energía Eléctrica de los clientes d...,https://www.datosabiertos.gob.pe/dataset/consu...,Empresa de Administración en Infraestructura E...,Consumo de Energía Eléctrica de los clientes d...,2023-10-09,NaN,Empresa de Administración en Infraestructura E...,655c11ea-25bb-4a46-ad12-01069f036d17,NaN,NaN,NaN,2023-10-09,NaN,Open Data Commons Attribution,Public,NaN,NaN
1,Consumo de Energía Eléctrica de los clientes d...,https://www.datosabiertos.gob.pe/dataset/consu...,Electro Ucayali,None,2022-11-09,Monthly,Electro Ucayali,3ba5c056-67db-41e5-aa9f-1197e743b1c2,Spanish (Peru),cesar.soria@electroucayali.com.pe,"Miércoles, Noviembre 9, 2022 - 00:00",2023-09-15,Soria Marina Cesar Homero,Open Data Commons Attribution,Public,NaN,Soria Marina Cesar Homero
2,GASTOS DE USO DE VEHÍCULOS DE LA EMPRESA ELECT...,https://www.datosabiertos.gob.pe/dataset/gasto...,Electro Ucayali,GASTOS DE USO DE VEHÍCULOS DE LA EMPRESA ELECT...,2023-06-05,Triennial,Electro Ucayali,b6006583-6ed8-4f2b-859c-aca1417a09a5,NaN,cesar.soria@electroucayali.com.pe,NaN,2023-09-15,Soria Marina Cesar Homero,Open Data Commons Attribution,Public,NaN,Soria Marina Cesar Homero
3,ORDENES DE COMPRA Y SERVICIOS DE LA EMPRESA EL...,https://www.datosabiertos.gob.pe/dataset/orden...,Electro Ucayali,Es el registro de órdenes de compra y servicio...,2023-05-31,NaN,Electro Ucayali,af9d03b3-a609-4ba7-98c1-646fbcfd1e9a,NaN,NaN,NaN,2023-09-15,NaN,Open Data Commons Attribution,Public,NaN,NaN
4,Registro de Hidrocarburos Líquidos - Setiembre...,https://www.datosabiertos.gob.pe/dataset/regis...,Presidencia del Consejo de Ministros,Para desarrollar actividades de comercializaci...,2023-09-12,NaN,Presidencia del Consejo de Ministros,47391ada-c355-40c6-b290-a88c3a56938f,NaN,NaN,NaN,2023-09-12,NaN,,Public,NaN,NaN


Most of the publishers of the datasets did not share their email addresses

In [19]:
df[["title", "Publisher", "Contact Email"]]

,title,Publisher,Contact Email
0,Consumo de Energía Eléctrica de los clientes d...,Empresa de Administración en Infraestructura E...,NaN
1,Consumo de Energía Eléctrica de los clientes d...,Electro Ucayali,cesar.soria@electroucayali.com.pe
2,GASTOS DE USO DE VEHÍCULOS DE LA EMPRESA ELECT...,Electro Ucayali,cesar.soria@electroucayali.com.pe
3,ORDENES DE COMPRA Y SERVICIOS DE LA EMPRESA EL...,Electro Ucayali,NaN
4,Registro de Hidrocarburos Líquidos - Setiembre...,Presidencia del Consejo de Ministros,NaN
5,Consumo de Energía Eléctrica de los clientes d...,NaN,NaN
6,Consumo energético de clientes Hidrandina [Dis...,Empresa Regional de Servicio Público de Electr...,hcalderonv@distriluz.com.pe
7,Consumo de Energía Eléctrica de los clientes d...,Empresa de Administración en Infraestructura E...,NaN
8,PADRÓN DE BENEFICIARIOS FISE POR LOCALIDAD DE ...,Electro Ucayali,NaN
9,Consumo de Energía Eléctrica de los clientes d...,Empresa de Administración en Infraestructura E...,NaN
